# imports

In [1]:
import pandas as pd
import numpy as np
import sklearn
import pickle 
import time 
import datetime
import warnings
warnings.filterwarnings('ignore')

In [17]:
df_train1 = pd.read_csv('~/Dropbox/Data/df_train1.csv')
df_train2 = pd.read_csv('~/Dropbox/Data/df_train2.csv')
df_train3 = pd.read_csv('~/Dropbox/Data/df_train3.csv')
df_train4 = pd.read_csv('~/Dropbox/Data/df_train4.csv')
df_train5 = pd.read_csv('~/Dropbox/Data/df_train5.csv')
df_train6 = pd.read_csv('~/Dropbox/Data/df_train6.csv')
df_train7 = pd.read_csv('~/Dropbox/Data/df_train7.csv')
df_train8 = pd.read_csv('~/Dropbox/Data/df_train8.csv')
df_test = pd.read_csv('~/Dropbox/Data/df_test.csv')

In [6]:
(df_train1.shape, df_train1.is_fraud.mean()), (df_test.shape, df_test.is_fraud.mean())

(((734003, 22), 0.005728859418830713), ((314572, 22), 0.005725239372862174))

In [18]:
_df1 = pd.concat([df_train1, df_test])
_df2 = pd.concat([df_train2, df_test])
_df3 = pd.concat([df_train3, df_test])
_df4 = pd.concat([df_train4, df_test])
_df5 = pd.concat([df_train5, df_test])
_df6 = pd.concat([df_train6, df_test])
_df7 = pd.concat([df_train7, df_test])
_df8 = pd.concat([df_train8, df_test])

In [19]:
_df1_mean = _df1.is_fraud.mean()
_df2_mean = _df2.is_fraud.mean()
_df3_mean = _df3.is_fraud.mean()
_df4_mean = _df4.is_fraud.mean()
_df5_mean = _df5.is_fraud.mean()
_df6_mean = _df6.is_fraud.mean()
_df7_mean = _df7.is_fraud.mean()
_df8_mean = _df8.is_fraud.mean()

In [14]:
def auto_amt_ver0503(df_tr, df_tst, _df_mean):
    df_tr = df_tr[["amt","is_fraud"]]
    df_tst = df_tst[["amt","is_fraud"]]     
    tr = TabularDataset(df_tr)
    tst = TabularDataset(df_tst)
    predictr = TabularPredictor(label="is_fraud", verbosity=0)
    t1 = time.time()
    predictr.fit(tr)
    t2 = time.time()
    time_diff = t2 - t1
    models = predictr._trainer.model_graph.nodes
    results = []
    for model_name in models:
    # 모델 평가
        eval_result = predictr.evaluate(tst, model=model_name)

    # 결과를 데이터프레임에 추가
        results.append({'model': model_name, 
                        'acc': eval_result['accuracy'], 
                        'pre': eval_result['precision'], 
                        'rec': eval_result['recall'], 
                        'f1': eval_result['f1'], 
                        'auc': eval_result['roc_auc']})
        
    model = []
    time_diff = []
    acc = []
    pre = []
    rec = []
    f1 = [] 
    auc = [] 
    graph_based = []
    method = [] 
    throw_rate = [] 
    train_size = []
    train_cols = []
    train_frate = []
    test_size = []
    test_frate = []
    hyper_params = [] 
    
    for result in results:
        model_name = result['model']
        model.append(model_name)
        time_diff.append(None)  # 각 모델별로 학습한 시간을 나타내고 싶은데 잘 안됨
        acc.append(result['acc']) 
        pre.append(result['pre'])
        rec.append(result['rec'])
        f1.append(result['f1'])
        auc.append(result['auc'])
        graph_based.append(False) 
        method.append('Autogluon') 
        throw_rate.append(_df_mean)
        train_size.append(len(tr))
        train_cols.append([col for col in tr.columns if col != 'is_fraud'])
        train_frate.append(tr.is_fraud.mean())
        test_size.append(len(tst))
        test_frate.append(tst.is_fraud.mean())
        hyper_params.append(None)
        
    df_results = pd.DataFrame(dict(
        model=model,
        time=time_diff,
        acc=acc,
        pre=pre,
        rec=rec,
        f1=f1,
        auc=auc,
        graph_based=graph_based,
        method=method,
        throw_rate=throw_rate,  
        train_size=train_size,
        train_cols=train_cols,
        train_frate=train_frate,
        test_size=test_size,
        test_frate=test_frate,
        hyper_params=hyper_params
    ))    
    ymdhms = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d-%H%M%S') 
    df_results.to_csv(f'../results2/{ymdhms}-Autogluon.csv',index=False)
    return df_results

In [16]:
auto_amt_ver0503(df_train1, df_test, _df1_mean)

,model,time,acc,pre,rec,f1,auc,graph_based,method,throw_rate,train_size,train_cols,train_frate,test_size,test_frate,hyper_params
0,KNeighborsUnif,None,0.994218,0.489865,0.241532,0.323540,0.782174,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None
1,KNeighborsDist,None,0.993582,0.403197,0.252082,0.310215,0.750937,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None
2,LightGBMXT,None,0.994275,0.000000,0.000000,0.000000,0.765047,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None
3,LightGBM,None,0.994275,0.000000,0.000000,0.000000,0.957828,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None
4,RandomForestGini,None,0.993140,0.363844,0.264853,0.306555,0.816185,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None
5,RandomForestEntr,None,0.993140,0.363844,0.264853,0.306555,0.816185,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None
6,CatBoost,None,0.994275,0.000000,0.000000,0.000000,0.870449,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None
7,ExtraTreesGini,None,0.993483,0.393316,0.254858,0.309299,0.840480,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None
8,ExtraTreesEntr,None,0.993493,0.393966,0.253748,0.308680,0.839967,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None
9,NeuralNetFastAI,None,0.994275,0.000000,0.000000,0.000000,0.943377,False,Autogluon,0.005728,734003,[amt],0.005729,314572,0.005725,None


In [20]:
auto_amt_ver0503(df_train2, df_test, _df2_mean)

,model,time,acc,pre,rec,f1,auc,graph_based,method,throw_rate,train_size,train_cols,train_frate,test_size,test_frate,hyper_params
0,KNeighborsUnif,None,0.993753,0.440837,0.339256,0.383433,0.816442,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None
1,KNeighborsDist,None,0.992984,0.378443,0.350916,0.364160,0.793232,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None
2,LightGBMXT,None,0.994275,0.000000,0.000000,0.000000,0.765797,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None
3,LightGBM,None,0.994275,0.000000,0.000000,0.000000,0.957293,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None
4,RandomForestGini,None,0.992393,0.341880,0.355358,0.348489,0.840338,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None
5,RandomForestEntr,None,0.992393,0.341880,0.355358,0.348489,0.840338,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None
6,CatBoost,None,0.994253,0.496150,0.250416,0.332841,0.936583,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None
7,ExtraTreesGini,None,0.992866,0.371296,0.354803,0.362862,0.860650,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None
8,ExtraTreesEntr,None,0.992911,0.373898,0.353137,0.363221,0.858746,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None
9,NeuralNetFastAI,None,0.993804,0.435315,0.276513,0.338200,0.929382,False,Autogluon,0.008171,420500,[amt],0.01,314572,0.005725,None


In [21]:
auto_amt_ver0503(df_train3, df_test, _df3_mean)

,model,time,acc,pre,rec,f1,auc,graph_based,method,throw_rate,train_size,train_cols,train_frate,test_size,test_frate,hyper_params
0,KNeighborsUnif,None,0.988111,0.269002,0.626874,0.376459,0.881531,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None
1,KNeighborsDist,None,0.986839,0.239242,0.595780,0.341394,0.866643,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None
2,LightGBMXT,None,0.994275,0.000000,0.000000,0.000000,0.781026,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None
3,LightGBM,None,0.990857,0.327004,0.564131,0.414018,0.962248,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None
4,RandomForestGini,None,0.985933,0.221444,0.579123,0.320381,0.892199,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None
5,RandomForestEntr,None,0.985933,0.221444,0.579123,0.320381,0.892199,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None
6,CatBoost,None,0.989109,0.289889,0.622432,0.395554,0.947419,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None
7,ExtraTreesGini,None,0.986935,0.240853,0.595780,0.343031,0.901395,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None
8,ExtraTreesEntr,None,0.986881,0.239937,0.595780,0.342101,0.900649,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None
9,NeuralNetFastAI,None,0.992202,0.358630,0.459189,0.402727,0.892828,False,Autogluon,0.015065,84100,[amt],0.05,314572,0.005725,None


In [22]:
auto_amt_ver0503(df_train4, df_test, _df4_mean)

,model,time,acc,pre,rec,f1,auc,graph_based,method,throw_rate,train_size,train_cols,train_frate,test_size,test_frate,hyper_params
0,KNeighborsUnif,None,0.982595,0.201106,0.686285,0.311061,0.900398,False,Autogluon,0.016841,42050,[amt],0.1,314572,0.005725,None
1,KNeighborsDist,None,0.978968,0.164834,0.657413,0.263580,0.881212,False,Autogluon,0.016841,42050,[amt],0.1,314572,0.005725,None
2,LightGBMXT,None,0.978984,0.161268,0.635758,0.257274,0.946402,False,Autogluon,0.016841,42050,[amt],0.1,314572,0.005725,None
3,LightGBM,None,0.988766,0.286629,0.646308,0.397134,0.961948,False,Autogluon,0.016841,42050,[amt],0.1,314572,0.005725,None
4,RandomForestGini,None,0.976641,0.146733,0.639645,0.238707,0.912443,False,Autogluon,0.016841,42050,[amt],0.1,314572,0.005725,None
5,RandomForestEntr,None,0.976641,0.146733,0.639645,0.238707,0.912443,False,Autogluon,0.016841,42050,[amt],0.1,314572,0.005725,None
6,CatBoost,None,0.984334,0.221946,0.692948,0.336207,0.954685,False,Autogluon,0.016841,42050,[amt],0.1,314572,0.005725,None
7,ExtraTreesGini,None,0.978558,0.161833,0.656857,0.259686,0.922153,False,Autogluon,0.016841,42050,[amt],0.1,314572,0.005725,None
8,ExtraTreesEntr,None,0.978565,0.162062,0.657968,0.260068,0.921562,False,Autogluon,0.016841,42050,[amt],0.1,314572,0.005725,None
9,NeuralNetFastAI,None,0.978072,0.169241,0.724042,0.274353,0.933527,False,Autogluon,0.016841,42050,[amt],0.1,314572,0.005725,None


In [23]:
auto_amt_ver0503(df_train5, df_test, _df5_mean)

,model,time,acc,pre,rec,f1,auc,graph_based,method,throw_rate,train_size,train_cols,train_frate,test_size,test_frate,hyper_params
0,KNeighborsUnif,None,0.963411,0.108540,0.747363,0.189551,0.923442,False,Autogluon,0.017896,21025,[amt],0.2,314572,0.005725,None
1,KNeighborsDist,None,0.956846,0.090896,0.726263,0.161571,0.897693,False,Autogluon,0.017896,21025,[amt],0.2,314572,0.005725,None
2,LightGBMXT,None,0.975347,0.154720,0.740700,0.255972,0.951622,False,Autogluon,0.017896,21025,[amt],0.2,314572,0.005725,None
3,LightGBM,None,0.982532,0.205704,0.716824,0.319673,0.961749,False,Autogluon,0.017896,21025,[amt],0.2,314572,0.005725,None
4,RandomForestGini,None,0.949843,0.078371,0.721266,0.141380,0.927522,False,Autogluon,0.017896,21025,[amt],0.2,314572,0.005725,None
5,RandomForestEntr,None,0.949843,0.078371,0.721266,0.141380,0.927522,False,Autogluon,0.017896,21025,[amt],0.2,314572,0.005725,None
6,CatBoost,None,0.981248,0.194635,0.725153,0.306897,0.962530,False,Autogluon,0.017896,21025,[amt],0.2,314572,0.005725,None
7,ExtraTreesGini,None,0.953333,0.085378,0.736258,0.153012,0.933086,False,Autogluon,0.017896,21025,[amt],0.2,314572,0.005725,None
8,ExtraTreesEntr,None,0.952920,0.084616,0.735702,0.151775,0.935070,False,Autogluon,0.017896,21025,[amt],0.2,314572,0.005725,None
9,NeuralNetFastAI,None,0.973888,0.148217,0.750139,0.247527,0.948571,False,Autogluon,0.017896,21025,[amt],0.2,314572,0.005725,None


In [24]:
auto_amt_ver0503(df_train6, df_test, _df6_mean)

,model,time,acc,pre,rec,f1,auc,graph_based,method,throw_rate,train_size,train_cols,train_frate,test_size,test_frate,hyper_params
0,KNeighborsUnif,None,0.940697,0.072537,0.794003,0.132931,0.933847,False,Autogluon,0.018278,14017,[amt],0.299993,314572,0.005725,None
1,KNeighborsDist,None,0.932499,0.063441,0.784009,0.117383,0.904577,False,Autogluon,0.018278,14017,[amt],0.299993,314572,0.005725,None
2,LightGBMXT,None,0.971243,0.136405,0.754581,0.231044,0.947721,False,Autogluon,0.018278,14017,[amt],0.299993,314572,0.005725,None
3,LightGBM,None,0.964120,0.113510,0.773459,0.197968,0.961942,False,Autogluon,0.018278,14017,[amt],0.299993,314572,0.005725,None
4,RandomForestGini,None,0.920498,0.053932,0.779012,0.100881,0.934945,False,Autogluon,0.018278,14017,[amt],0.299993,314572,0.005725,None
5,RandomForestEntr,None,0.920498,0.053932,0.779012,0.100881,0.934945,False,Autogluon,0.018278,14017,[amt],0.299993,314572,0.005725,None
6,CatBoost,None,0.969501,0.129434,0.755691,0.221013,0.959693,False,Autogluon,0.018278,14017,[amt],0.299993,314572,0.005725,None
7,ExtraTreesGini,None,0.925063,0.057803,0.790117,0.107726,0.940287,False,Autogluon,0.018278,14017,[amt],0.299993,314572,0.005725,None
8,ExtraTreesEntr,None,0.925763,0.058505,0.792893,0.108970,0.940250,False,Autogluon,0.018278,14017,[amt],0.299993,314572,0.005725,None
9,NeuralNetFastAI,None,0.972080,0.139732,0.751805,0.235663,0.898952,False,Autogluon,0.018278,14017,[amt],0.299993,314572,0.005725,None


In [25]:
auto_amt_ver0503(df_train7, df_test, _df7_mean)

,model,time,acc,pre,rec,f1,auc,graph_based,method,throw_rate,train_size,train_cols,train_frate,test_size,test_frate,hyper_params
0,KNeighborsUnif,None,0.914633,0.053725,0.837313,0.100971,0.939006,False,Autogluon,0.018475,10512,[amt],0.400019,314572,0.005725,None
1,KNeighborsDist,None,0.905713,0.048228,0.825652,0.091132,0.911215,False,Autogluon,0.018475,10512,[amt],0.400019,314572,0.005725,None
2,LightGBMXT,None,0.945599,0.080585,0.816768,0.146697,0.955586,False,Autogluon,0.018475,10512,[amt],0.400019,314572,0.005725,None
3,LightGBM,None,0.934616,0.069347,0.838978,0.128105,0.961903,False,Autogluon,0.018475,10512,[amt],0.400019,314572,0.005725,None
4,RandomForestGini,None,0.887406,0.040562,0.823987,0.077318,0.938095,False,Autogluon,0.018475,10512,[amt],0.400019,314572,0.005725,None
5,RandomForestEntr,None,0.887406,0.040562,0.823987,0.077318,0.938095,False,Autogluon,0.018475,10512,[amt],0.400019,314572,0.005725,None
6,CatBoost,None,0.946769,0.082009,0.813992,0.149006,0.958300,False,Autogluon,0.018475,10512,[amt],0.400019,314572,0.005725,None
7,ExtraTreesGini,None,0.897276,0.045003,0.837868,0.085418,0.944199,False,Autogluon,0.018475,10512,[amt],0.400019,314572,0.005725,None
8,ExtraTreesEntr,None,0.896577,0.044737,0.838423,0.084941,0.943396,False,Autogluon,0.018475,10512,[amt],0.400019,314572,0.005725,None
9,NeuralNetFastAI,None,0.970783,0.134520,0.755136,0.228360,0.901838,False,Autogluon,0.018475,10512,[amt],0.400019,314572,0.005725,None


In [26]:
auto_amt_ver0503(df_train8, df_test, _df8_mean)

,model,time,acc,pre,rec,f1,auc,graph_based,method,throw_rate,train_size,train_cols,train_frate,test_size,test_frate,hyper_params
0,KNeighborsUnif,None,0.879996,0.040024,0.868406,0.076520,0.938083,False,Autogluon,0.018595,8410,[amt],0.5,314572,0.005725,None
1,KNeighborsDist,None,0.876337,0.038557,0.860633,0.073808,0.907000,False,Autogluon,0.018595,8410,[amt],0.5,314572,0.005725,None
2,LightGBMXT,None,0.883591,0.041820,0.882288,0.079855,0.952322,False,Autogluon,0.018595,8410,[amt],0.5,314572,0.005725,None
3,LightGBM,None,0.908285,0.052093,0.873404,0.098322,0.960795,False,Autogluon,0.018595,8410,[amt],0.5,314572,0.005725,None
4,RandomForestGini,None,0.860633,0.034338,0.860633,0.066040,0.939192,False,Autogluon,0.018595,8410,[amt],0.5,314572,0.005725,None
5,RandomForestEntr,None,0.860633,0.034338,0.860633,0.066040,0.939192,False,Autogluon,0.018595,8410,[amt],0.5,314572,0.005725,None
6,CatBoost,None,0.900090,0.048243,0.878401,0.091464,0.961486,False,Autogluon,0.018595,8410,[amt],0.5,314572,0.005725,None
7,ExtraTreesGini,None,0.866968,0.036225,0.868406,0.069548,0.942842,False,Autogluon,0.018595,8410,[amt],0.5,314572,0.005725,None
8,ExtraTreesEntr,None,0.868199,0.036466,0.866185,0.069985,0.942820,False,Autogluon,0.018595,8410,[amt],0.5,314572,0.005725,None
9,NeuralNetFastAI,None,0.966326,0.118535,0.758468,0.205028,0.898449,False,Autogluon,0.018595,8410,[amt],0.5,314572,0.005725,None
